In [2]:
import numpy as np
import sys
import math
from matplotlib.pyplot import cm
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# plt.rcParams.update({'font.size': 20,'figure.figsize': (10.0, 6.0)})
import ipywidgets as wg
from ipywidgets import Layout
# %matplotlib widget
# import PyQt5
%matplotlib qt

import time
import csv
import matplotlib
from scipy.signal import savgol_filter

def forceAspect(ax,aspect):
    im = ax.get_images()
    extent =  im[0].get_extent()
    ax.set_aspect(abs((extent[1]-extent[0])/(extent[3]-extent[2]))/aspect)
    
    

In [3]:
#Parameters

#Dunker
D1=1.27e-2                 #Inner Diameter of Dunker in m
D2=1.397e-2                #Outer Diameter of Dunker in m
Area_SS=(np.pi*(D2/2)**2)-(np.pi*(D1/2)**2)  #Area of cross section of stainless steel 
Temp2=4                    #Final Temp in K
Temp1=300                  #Initial Temp in K
TempDiff= Temp1-Temp2      #Final Change in Temp 

#wiring
downwiring=2               #Length of downward wiring in m
upwiring=2                 #Length of upward wiring in m
totalwiring= 4             #Total length in m 
D=0.127e-3                 #Diameter of a 36AWG wire
Area_wire=np.pi*(D/2)**2   #Area of cross section of a 36AWG wire
no_wires=4

#current
I=10e-3                    #Operating Current in A

#Iterations
num=500

In [4]:
## Wiring
#12-pair Copper Loom - 4m

ResPerM_Cu= 2.3 #Electrical Resistance per meter ohm/m
PtPerMK_Cu= 345 #Thermal conductivity in W/m.k
#source: https://cds.cern.ch/record/1973682/files/arXiv:1501.07100.pdf


#Thermal Conductivity
Pt_Cu=no_wires*(PtPerMK_Cu*Area_wire*TempDiff)/totalwiring #total thermal power

#Electrical Conductivity
R_Cu= ResPerM_Cu*totalwiring
# print(R_Cu)
Pe_Cu= R_Cu*(I)**2 #total electrical power in W
 
print('Copper Loom:','\nElectrical Power=',Pe_Cu*1e3,'mW','\nThermal Power=',Pt_Cu*1e3,'mW')

Copper Loom: 
Electrical Power= 0.9199999999999999 mW 
Thermal Power= 1.2936241941359117 mW


In [5]:
##Wiring
# 12-pair Beryllium-Copper Loom - 4m

ResPerM_Be_Cu= 10 #Electrical Resistance per meter ohm/m
PtPerMK_Be_Cu= 120  # Average Thermal conductivity in W/m.k

#Thermal Conductivity
Pt_Be_Cu=no_wires*(PtPerMK_Be_Cu*Area_wire*TempDiff)/totalwiring #total thermal power in W

#Electrical Conductivity
R_Be_Cu= ResPerM_Be_Cu*totalwiring  
Pe_Be_Cu= R_Be_Cu*(I)**2 #total electrical power in W

print('Beryllium-Copper Loom:','\nElectrical Power=',Pe_Be_Cu*1e3,'mW','\nThermal Power=',Pt_Be_Cu*1e3,'mW')

Beryllium-Copper Loom: 
Electrical Power= 4.0 mW 
Thermal Power= 0.44995624143857793 mW


In [17]:
#Boil-Off Calculation For Wiring
Enthalpy_He= 0.0845 # Enthalpy of Vaporization of Helium 
Dens_LHe=125 #Density of Liquid Helium in g/L
MolMass_He=4 #Molar Mass of Helium in g/mol 
TotMol_He=Dens_LHe/MolMass_He
boilrate_He=Enthalpy_He*TotMol_He #How much Joules it takes to boil 1 Liter of He (kJ/L)


TimePerLiter_Cu= (boilrate_He*1e3/(Pt_Cu+Pe_Cu))/60
TimePerLiter_Be_Cu= (boilrate_He*1e3/(Pt_Be_Cu+Pe_Be_Cu))/60


print('Time to boil 1 L of He','\nCu -->',TimePerLiter_Cu,'min (or)',TimePerLiter_Cu/(60*24),'days'\
      , '\nBe-Cu -->',TimePerLiter_Be_Cu,'min (or)',TimePerLiter_Be_Cu/(60*24),'days')

Time to boil 1 L of He 
Cu --> 19881.611695089974 min (or) 13.806674788256926 days 
Be-Cu --> 9890.078526354007 min (or) 6.868110087745839 days


# Boil-Off Calculations for the Sample Holder 

## Specific heat Change with Temp for Cu

Interpolated from: 

![](c_Cu1.png "c_Cu") 


In [7]:
# Copper box (Karim's)

ro=8.9e6          #Density in g/m3
vol=7.57e-6       #vol in m3


#Mass
m_Cu=ro*vol        #Assuming mass of 150g

##Specific Heat c

#Interpolation to get c 

w = []
x = []
y = []
    
with open('Copper.csv', newline='') as f:
    reader = csv.reader(f, delimiter=',')
    
    for row in reader:
            w.extend(row)
    
           
for i in range(len(w)):
    if i%2==1:
        y.append(float(w[i]))
    else:
        x.append(float(w[i]))
        
# print(y)
z=np.linspace(0,300,num)
c_Cu= np.interp(z, x, y)*1e-3
# print(c_Cu)
# plt.plot(z,c_Cu)

dT=300/num 
# TempDiff=np.subtract(300,z[::-1])

# print(TempDiff)

#Thermal Energy
Q_Cu=m_Cu*sum(np.multiply(c_Cu[::-1],dT))
# print(Q_Cu)
# print('Cu Box:''\nThermal Energy=',Q_Cu,'kJ')

## Specific heat Change with Temp for G10
Interpolated from: 

http://www.yutopian.com/Yuan/prop/G10.html

In [8]:
# G10 box (based on 67g of Cu)


## mass
m_G10= (m_Cu*2)/9          #Ratio of Densities is 2/9


##Specific Heat c

#Interpolation to get c 

w = []
x = []
y = []
    
with open('G10.csv', newline='') as f:
    reader = csv.reader(f, delimiter=',')
    
    for row in reader:
            w.extend(row)
    
           
for i in range(len(w)):
    if i%2==1:
        y.append(float(w[i]))
    else:
        x.append(float(w[i]))

x=x[::-1]
y=y[::-1]


z=np.linspace(0,300,num)
c_G10= np.interp(z, x, y)
# print(c_G10)
# TempDiff=np.subtract(300,z[::-1])

# plt.plot(x,y)

#Thermal Energy
Q_G10=m_G10*sum(np.multiply(c_G10[::-1],dT)) #Energy in J
# print('G10 Box:''\nThermal Energy=',Q_G10,'kJ') 

In [9]:
#Boil-Off Calculation
Enthalpy_He= 0.0845 # Enthalpy of Vaporization of Helium 
Dens_LHe=125 #Density of Liquid Helium in g/L
MolMass_He=4 #Molar Mass of Helium in g/mol 
TotMol_He=Dens_LHe/MolMass_He
boilrate_He=Enthalpy_He*TotMol_He #How much Joules it takes to boil 1 Liter of He (kJ/L)


NoOfLiters_Cu=Q_Cu/(boilrate_He*1e3)
NoOfLiters_G10= Q_G10/(boilrate_He*1e3)

# print(NoOfLiters_Cu)
# print(NoOfLiters_G10)


print('Mass of', m_Cu, 'g of Cu gives:\n\nNumber of LHe Liters needed to cool down from 300 to 0: \nG10 -->',NoOfLiters_G10,'L', '\nCu -->',NoOfLiters_Cu,'L \n\nSo Relative Time to boil 1 L of LHe is\nT_G10='\
      ,NoOfLiters_Cu/NoOfLiters_G10, 'T_Cu\nAssuming same cooling power.')

Mass of 67.373 g of Cu gives:

Number of LHe Liters needed to cool down from 300 to 0: 
G10 --> 0.8340543380301555 L 
Cu --> 2.063786097267379 L 

So Relative Time to boil 1 L of LHe is
T_G10= 2.4744024497751154 T_Cu
Assuming same cooling power.


In [55]:
# ## Color Map
# # print(c_G10)
# T=np.zeros((num),dtype=np.float)
# m_Cu=np.linspace(50,300,num)*1e-3
# m_G10=np.zeros((num),dtype=np.float)
# Q_Cu=np.zeros((num,num),dtype=np.float)
# Q_G10=np.zeros((num,num),dtype=np.int)
# NoOfLiters_Cu=np.zeros((num,num),dtype=np.float)
# NoOfLiters_G10=np.zeros((num,num),dtype=np.float)

# for i in range(num): #Temp
#     T[i]=300-i
#     for k in range(num): #mass
#         m_G10[k]=m_Cu[k]*2/9        
#         Q_Cu[k,i]=m_Cu[k]*c_Cu[i]*(T[i]-300)
#         Q_G10[k,i]=m_G10[k]*c_G10[i]*(T[i]-300)
#         NoOfLiters_Cu[k,i]=Q_Cu[k,i]/(boilrate_He*1e3)
#         NoOfLiters_G10[k,i]= Q_G10[k,i]/(boilrate_He*1e3)
    

# #Plotting Cu     
# fig= plt.figure(1)
# ax = fig.add_subplot(111)
# NoOfLiters_Cu=NoOfLiters_Cu-NoOfLiters_Cu.min()
# img = plt.imshow(NoOfLiters_Cu,extent= [min(T),max(T),min(m_Cu)*1e3,max(m_Cu)*1e3],cmap='viridis_r')
# forceAspect(ax,aspect=1.0)
# plt.colorbar()
# plt.xlabel('$T$ [K]')
# plt.ylabel('$Mass$ [g]')
# plt.title('Cu\nNumber of Liters VS. $m&T$')

# #Plotting G10      
# fig2= plt.figure(2)
# ax = fig2.add_subplot(111)
# NoOfLiters_G10=NoOfLiters_G10-NoOfLiters_G10.min()
# img = plt.imshow(NoOfLiters_G10,extent= [min(z),max(z),min(m_G10)*1e3,max(m_G10)*1e3],cmap='viridis_r')
# forceAspect(ax,aspect=1.0)
# plt.colorbar()
# plt.xlabel('$T$ [K]')
# plt.ylabel('$Mass$ [g]')
# plt.title('G10\nNumber of Liters VS. $m&T$')


<ipython-input-55-9bbda2a3007e>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  T=np.zeros((num),dtype=np.float)
<ipython-input-55-9bbda2a3007e>:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  m_G10=np.zeros((num),dtype=np.float)
<ipython-input-55-9bbda2a3007e>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `flo

Text(0.5, 1.0, 'G10\nNumber of Liters VS. $m&T$')